In [3]:
#import dependencies
import pandas as pd
import numpy as np
import requests
import json
import random
import matplotlib.pyplot as plt


In [4]:
#path to CSV file
csv_path="2015_census_tract_data.csv"

#read CSV file
Census_file=pd.read_csv(csv_path)


In [5]:
#check to see if there are rows with missing data
Census_file.count()

CensusTract        74001
State              74001
County             74001
TotalPop           74001
Men                74001
Women              74001
Hispanic           73311
White              73311
Black              73311
Native             73311
Asian              73311
Pacific            73311
Citizen            74001
Income             72901
IncomeErr          72901
IncomePerCap       73261
IncomePerCapErr    73261
Poverty            73166
ChildPoverty       72883
Professional       73194
Service            73194
Office             73194
Construction       73194
Production         73194
Drive              73204
Carpool            73204
Transit            73204
Walk               73204
OtherTransp        73204
WorkAtHome         73204
MeanCommute        73052
Employed           74001
PrivateWork        73194
PublicWork         73194
SelfEmployed       73194
FamilyWork         73194
Unemployment       73199
dtype: int64

In [6]:
#remove rows with missing data
clean_census_file=Census_file.dropna(how="any")
clean_census_file.count()

CensusTract        72727
State              72727
County             72727
TotalPop           72727
Men                72727
Women              72727
Hispanic           72727
White              72727
Black              72727
Native             72727
Asian              72727
Pacific            72727
Citizen            72727
Income             72727
IncomeErr          72727
IncomePerCap       72727
IncomePerCapErr    72727
Poverty            72727
ChildPoverty       72727
Professional       72727
Service            72727
Office             72727
Construction       72727
Production         72727
Drive              72727
Carpool            72727
Transit            72727
Walk               72727
OtherTransp        72727
WorkAtHome         72727
MeanCommute        72727
Employed           72727
PrivateWork        72727
PublicWork         72727
SelfEmployed       72727
FamilyWork         72727
Unemployment       72727
dtype: int64

In [7]:
#column names
clean_census_file.columns

Index(['CensusTract', 'State', 'County', 'TotalPop', 'Men', 'Women',
       'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [8]:
#create a dataframe from the file
DF_census_file=pd.DataFrame(clean_census_file)
DF_census_file.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


In [9]:
grouped_income = DF_census_file.groupby(["State","County"]).apply(lambda x: np.average(x["Income"],weights=x["TotalPop"]))
grouped_income=pd.DataFrame(grouped_income,columns=["Income"])
grouped_income=grouped_income.reset_index()
grouped_income["ID"] = grouped_income["State"]+grouped_income["County"]
grouped_income.head()

,State,County,Income,ID
0,Alabama,Autauga,51696.286576,AlabamaAutauga
1,Alabama,Baldwin,51074.359577,AlabamaBaldwin
2,Alabama,Barbour,32959.300349,AlabamaBarbour
3,Alabama,Bibb,38886.634888,AlabamaBibb
4,Alabama,Blount,46237.969018,AlabamaBlount


In [10]:
#find total number of people per ethnicity
DF_census_file["Hispanic Population"]=(DF_census_file["Hispanic"]/100)*DF_census_file["TotalPop"]
DF_census_file["White Population"]=(DF_census_file["White"]/100)*DF_census_file["TotalPop"]
DF_census_file["Black Population"]=(DF_census_file["Black"]/100)*DF_census_file["TotalPop"]
DF_census_file["Native Population"]=(DF_census_file["Native"]/100)*DF_census_file["TotalPop"]
DF_census_file["Asian Population"]=(DF_census_file["Asian"]/100)*DF_census_file["TotalPop"]
DF_census_file["Pacific Population"]=(DF_census_file["Pacific"]/100)*DF_census_file["TotalPop"]
DF_census_file["Other Population"]=DF_census_file["TotalPop"]-DF_census_file["Pacific Population"]-DF_census_file["Asian Population"]-DF_census_file["Native Population"]-DF_census_file["Hispanic Population"]-DF_census_file["White Population"]-DF_census_file["Black Population"]
DF_census_file["ID"]=DF_census_file["State"]+DF_census_file["County"]


In [11]:
TractCount=DF_census_file.groupby(["State","County"]).count()["CensusTract"]
TractCount=pd.DataFrame(TractCount)
TractCount=TractCount.reset_index()
TractCount.head()

,State,County,CensusTract
0,Alabama,Autauga,12
1,Alabama,Baldwin,31
2,Alabama,Barbour,9
3,Alabama,Bibb,4
4,Alabama,Blount,9


In [12]:
DF_census_file.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,FamilyWork,Unemployment,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Pacific Population,Other Population,ID
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.0,5.4,17.532,1702.552,149.996,5.844,11.688,0.000,60.388,AlabamaAutauga
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,13.3,17.248,871.024,1149.148,0.000,49.588,0.000,68.992,AlabamaAutauga
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,6.2,0.000,2211.160,552.048,14.840,41.552,8.904,139.496,AlabamaAutauga
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,10.8,464.415,3662.244,163.651,70.768,0.000,0.000,61.922,AlabamaAutauga
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,4.2,75.341,7372.655,2669.224,0.000,408.994,0.000,236.786,AlabamaAutauga


In [13]:
Test_census_file=DF_census_file[["State","County","ID","TotalPop","Men","Women","Hispanic Population","White Population","Black Population","Native Population","Asian Population","Other Population"]]
Test_census_file=Test_census_file.groupby(["State","County","ID"],as_index=False).sum()
Test_census_file.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314


In [14]:
merge_table=pd.merge(Test_census_file,grouped_income,on=["State","County","ID"] ,how="outer")
merge_table.head()


,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population,Income
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898,51696.286576
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128,51074.359577
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398,32959.300349
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569,38886.634888
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314,46237.969018


In [15]:
merge_table=pd.merge(merge_table,TractCount,on=["State","County"],how="outer")
merge_table.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population,Income,CensusTract
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898,51696.286576,12
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128,51074.359577,31
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398,32959.300349,9
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569,38886.634888,4
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314,46237.969018,9


In [16]:
#Filter
merge_table["Hispanic Population %"]=merge_table["Hispanic Population"]/merge_table["TotalPop"]*100
merge_table["White Population %"]=merge_table["White Population"]/merge_table["TotalPop"]*100
merge_table["Black  Population %"]=merge_table["Black Population"]/merge_table["TotalPop"]*100
merge_table["Asian Population %"]=merge_table["Asian Population"]/merge_table["TotalPop"]*100
merge_table["Other Population %"]=merge_table["Other Population"]/merge_table["TotalPop"]*100
merge_table["Native Population %"]=merge_table["Native Population"]/merge_table["TotalPop"]*100
merge_table["% Men"]=merge_table["Men"]/merge_table["TotalPop"]*100
merge_table["% Women"]=merge_table["Women"]/merge_table["TotalPop"]*100


In [17]:
merge_table.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,...,Income,CensusTract,Hispanic Population %,White Population %,Black Population %,Asian Population %,Other Population %,Native Population %,% Men,% Women
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,...,51696.286576,12,2.614243,75.788227,18.526968,0.957654,1.674903,0.421881,48.432661,51.567339
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,...,51074.359577,31,4.493247,83.102616,9.496849,0.664696,1.683124,0.559468,48.848663,51.151337
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,...,32959.300349,9,4.628312,46.231594,46.690625,0.436741,1.824588,0.188141,53.828160,46.171840
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,...,38886.634888,4,2.209158,74.499889,21.425137,0.100606,1.334140,0.431070,53.410901,46.589099
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,...,46237.969018,9,8.634815,87.853854,1.520305,0.148449,1.551402,0.291175,49.405649,50.594351


In [18]:
merged_clean=merge_table[["State","County","TotalPop","Income","CensusTract","White Population %","Hispanic Population %",
                         "Black  Population %","Asian Population %", "Native Population %","Other Population %", "% Men", "% Women"]]

In [19]:
merged_clean.head()

,State,County,TotalPop,Income,CensusTract,White Population %,Hispanic Population %,Black Population %,Asian Population %,Native Population %,Other Population %,% Men,% Women
0,Alabama,Autauga,55221,51696.286576,12,75.788227,2.614243,18.526968,0.957654,0.421881,1.674903,48.432661,51.567339
1,Alabama,Baldwin,195121,51074.359577,31,83.102616,4.493247,9.496849,0.664696,0.559468,1.683124,48.848663,51.151337
2,Alabama,Barbour,26932,32959.300349,9,46.231594,4.628312,46.690625,0.436741,0.188141,1.824588,53.828160,46.171840
3,Alabama,Bibb,22604,38886.634888,4,74.499889,2.209158,21.425137,0.100606,0.431070,1.334140,53.410901,46.589099
4,Alabama,Blount,57710,46237.969018,9,87.853854,8.634815,1.520305,0.148449,0.291175,1.551402,49.405649,50.594351


In [20]:
#filter for Counties with Census Tracts less than 30
Large_County=merged_clean.query("TotalPop>=750000")
#small_county = small_county.query("TotalPop>=200000")
Large_County.to_csv("Large_County.csv")